In [ ]:
# This will reconstruct the Aluminum folders for the DECT scans

import os
import numpy as np
from reconstruction.ring_corr import devon_correction
from reconstruction.find_dead_pixels import find_dead_pixels
from reconstruction.inidividual_bin_recon import correct_dead_pixels, reconstruct_CT

directory = r'\Material_decomposition_data/Calibration_and_test_data'
folders = ['23_11_22_CT_Al_80', '23_11_22_CT_Al_120']
peak_energies = [80, 120]
sub_folders = ['both']

# VARIABLES TO CHANGE
save = True

num_angles = 720  # The number of projections
scan_duration = 180  # The length of the CT scan (s)
offset = -1

proj_time = scan_duration/num_angles  # The scale factor (the scan time per projection/angle)

for lidx, load_folder in enumerate(folders):

    # Load the raw flat field, dark field, and water phantom scans (Only the full counts bin)
    air60 = np.load(os.path.join(directory, load_folder, 'airscan_65s', 'Data', 'data.npy'))[1:, :, :, -1]
    air1 = np.sum(air60, axis=0) / (60 / proj_time)  # Sum all frames to get the full 60s flat field scan (scaled correctly per frame)
    dark = np.load(os.path.join(directory, load_folder, 'darkscan_60s', 'Data', 'data.npy'))[:, :, -1]
    water = np.load(os.path.join(directory, load_folder, 'water', 'Data', 'data.npy'))[1:, :, :, -1]
    water = np.sum(water, axis=0)  # Sum all frames

    # Go through each sub folder
    for sub in sub_folders:

        data = np.load(os.path.join(directory, load_folder, sub, 'Data', 'data.npy'))[:, :, :, -1]

         # This will cut the projections down to the correct number if there are more than necessary
        if num_angles != len(data):
            diff = abs(num_angles - len(data))
            data = data[int(np.ceil(diff / 2)):len(data) - diff // 2]

        save_file = f'25-{peak_energies[lidx]}.npy'

        os.makedirs(os.path.join(directory, load_folder, sub, 'CT'), exist_ok=True)

        # The save locations for the sinogram, CT, and normalized CT
        sino_save_path = os.path.join(directory, load_folder, sub, 'Data', save_file)
        ct_save_path = os.path.join(directory, load_folder, sub, 'CT', f'CT_{save_file}')

        print(sub, save_file)

        # Create the dead pixel mask
        dpm = find_dead_pixels(np.sum(air60[0:6], axis=0), np.sum(air60[6:], axis=0), dark)

        # Correct raw data and air data for dead pixels
        print('Correcting dead pixels in raw data')
        data = correct_dead_pixels(data, dpm, True, False)
        print('Correcting dead pixels in air scan')
        air1 = correct_dead_pixels(air1, dpm, True, True)

        # Do the -ln(I/I0) correction
        sino = np.squeeze(np.log(air1 + 0.01) - np.log(data + 0.01))

        air1 = np.squeeze(air1)

        # Correct for pixel non-uniformities (ring artifacts)
        corr_array = devon_correction(water, air1 * (60 / proj_time), dpm, num_bins=1)

        # Correct the sinogram
        sino = np.multiply(corr_array, sino)

        # Save the sinogram and display it
        if save:
            np.save(sino_save_path, sino)

        # Now reconstruct the CT scan
         # Reconstruct the sinogram
        ct = reconstruct_CT(sino, h_offset=offset)

        # Save the CT scan
        if save:
            np.save(ct_save_path, ct)

In [ ]:
## This cell will normalize the CT scans to HU

import os
import numpy as np
from reconstruction.inidividual_bin_recon import normalize_ct
from glob import glob

directory = r'\Material_decomposition_data/Calibration_and_test_data'
folders = ['23_11_22_CT_Al_80', '23_11_22_CT_Al_120']

# VARIABLES TO CHANGE
save = True

# The sub folder names for each of the 3 CT acquisitions for the 3 sets of tissue equivalent cylinders
sub_folders = ['both']

# Go through each folder
for load_folder in folders:

    # Go through each sub folder
    for sub in sub_folders:

        if os.path.exists(os.path.join(directory, load_folder, sub)):

            ct_files = glob(os.path.join(directory, load_folder, sub, 'CT', '*25*'))

            for file in ct_files:

                # Set the path to load the CT scan
                load_path = file

                # Set the file and only the file's name
                file = file.split('\\')[-1]
                print(sub, file)

                os.makedirs(os.path.join(directory, load_folder, sub, 'Norm CT'), exist_ok=True)

                # The save locations for the CT
                norm_save_path = os.path.join(directory, load_folder, sub, 'Norm CT', f'Norm_{file}')

                # Load the CT scan
                ct = np.load(load_path)
                water_mask = np.load(os.path.join(directory, load_folder, sub, 'water_mask.npy'))

                ct = normalize_ct(ct, water_mask=water_mask, water_slice=16)

                # Save the CT scan
                if save:
                    np.save(norm_save_path, ct)

In [ ]:
# This will take all of the normalized data and the calibrated values and calculate the Zeff and electron density maps for DECT

import os
import numpy as np
from decomposition.bourque import solve_for_z, solve_for_rho
from datetime import datetime

directory = r'\Material_decomposition_data/Calibration_and_test_data'

folder_low = '23_11_22_CT_Al_80'
folder_high = '23_11_22_CT_Al_120'

low_t1, low_t2 = 25, 80
high_t1, high_t2 = 25, 120
K, M = 2, 3

roll = -1

file_low = f'Norm_CT_{low_t1}-{low_t2}_roll_{roll}.npy'
file_high = f'Norm_CT_{high_t1}-{high_t2}.npy'

# The sub folder names for each of the 3 CT acquisitions for the 3 sets of tissue equivalent cylinders
sub_folders = ['both']

calib_path = os.path.join(directory, '23_08_26_CT_inserts_80', 'Calibrated_values')
c = np.load(os.path.join(calib_path, f'C_{low_t1}-{low_t2}_{high_t1}-{high_t2}_K{K}.npy'))[0]
b_low, b_high = np.load(os.path.join(calib_path, f'B_{low_t1}-{low_t2}_{high_t1}-{high_t2}_M{M}.npy'))

start = datetime.now().timestamp()

for sidx, sub in enumerate(sub_folders):

    low_data = np.load(os.path.join(directory, folder_low, sub, 'Norm CT', file_low))
    high_data = np.load(os.path.join(directory, folder_high, sub, 'Norm CT', file_high))

    # Flatten the data arrays
    low_data = np.array(low_data.flatten())
    high_data = np.array(high_data.flatten())

    # Calculate the Zeff and rho maps for the whole image set
    z_calc = solve_for_z(low_data, high_data, c)
    rho_calc = solve_for_rho(z_calc, low_data, high_data, b_low, b_high)

    # Reshape the data into the original format
    z_calc = np.reshape(z_calc, (24, 512, 512))
    rho_calc = np.reshape(rho_calc, (24, 512, 512))

    # Save the data in the appropriate location
    mat_decomp_path = os.path.join(directory, folder_low, sub, 'Decomposition')
    os.makedirs(mat_decomp_path, exist_ok=True)

    np.save(os.path.join(mat_decomp_path, f'Z_map_K{K}_M{M}_roll_{roll}.npy'), z_calc)
    np.save(os.path.join(mat_decomp_path, f'rho_map_K{K}_M{M}_roll_{roll}.npy'), rho_calc)

print(datetime.now().timestamp() - start)
